In [9]:
import numpy as np
import os
import pandas as pd
import json
import sys
import regex as re
import openai
import time

In [10]:
DATA_FOLDER = "../data"

In [11]:
def load_file(fname):
    with open(os.path.join(DATA_FOLDER, fname), 'r') as f:
        return json.load(f)

In [12]:
test = load_file("test.json")
train = load_file("train.json")
val = load_file("val.json")

In [13]:
test[0]

{'context': 'Negotiations between the union and the trucking industry will not resume unless one or the other of these parties agrees to make concessions. The trucking industry will not make any concessions without certain prior concessions being made by the union, and the union will make concessions only if the government agrees to act as mediator.',
 'question': 'If the statements above are true, which one of the following must also be true on the basis of them?',
 'answers': ['If the union makes concessions, then the trucking industry will also make concessions.',
  'Differences between the union and the trucking industry will be successfully negotiated if the government agrees to act as mediator.',
  'Negotiations between the union and the trucking industry will not resume.',
  'Negotiations between the union and the trucking industry will not resume unless the government agrees to act as mediator.'],
 'question_type': 5,
 'id_string': 'test_0'}

In [14]:
train[0]

{'context': "In rheumatoid arthritis, the body' s immune system misfunctions by attacking healthy cells in the joints causing the release of a hormone that in turn causes pain and swelling. This hormone is normally activated only in reaction to injury or infection. A new arthritis medication will contain a protein that inhibits the functioning of the hormone that causes pain and swelling in the joints.",
 'question': 'The statements above, if true, most strongly support which one of the following conclusions?',
 'answers': ['Unlike aspirin and other medications that reduce pain and swelling and that are currently available, the new medication would repair existing cell damage that had been caused by rheumatoid arthritis.',
  'A patient treated with the new medication for rheumatoid arthritis could sustain a joint injury without becoming aware of it.',
  'Joint diseases other than rheumatoid arthritis would not be affected by the new medication.',
  "The benefits to rheumatoid arthritis

In [7]:
for td in train[:100]:
    print(td['id_string'], td['question'])

train_0 The statements above, if true, most strongly support which one of the following conclusions?
train_1 The patient's argument proceeds by
train_2 The pattern of reasoning displayed above most closely parallels which of the following?
train_3 The argument's conclusion follows logically if which one of the following is assumed?
train_4 If the shipping clerk's statements are true, which of the following must also be true?
train_5 If, on the basis of the evidence above, it is argued that PRP contracts increase worker productivity, which of the following, if true, would most seriously weaken that argument?
train_6 Which one of the following, if true, most weakens the astronomer's argument?
train_7 Which one of the following, if true, most strengthens the argument?
train_8 Each of the following, if true, could explain the decrease described above EXCEPT:
train_9 If the statements above are true, which one of the following must also be true?
train_10 Which one of the following, if true,

In [15]:
train_ixs = [0, 1, 2, 3, 6, 7, 8, 12, 16, 20, 22, 32]

In [19]:
Options = ['A', 'B', 'C', 'D']
for ix in train_ixs:
    print('Context:',train[ix]['context'])
    print('Question:',train[ix]['question'])
    print('Answer Options:')
    for i, opt in enumerate(train[ix]['answers']):
        print(Options[i], ':', opt)
    
    print('Correct Answer:', Options[train[ix]['label']])
    print('<stop>')

Context: In rheumatoid arthritis, the body' s immune system misfunctions by attacking healthy cells in the joints causing the release of a hormone that in turn causes pain and swelling. This hormone is normally activated only in reaction to injury or infection. A new arthritis medication will contain a protein that inhibits the functioning of the hormone that causes pain and swelling in the joints.
Question: The statements above, if true, most strongly support which one of the following conclusions?
Answer Options:
A : Unlike aspirin and other medications that reduce pain and swelling and that are currently available, the new medication would repair existing cell damage that had been caused by rheumatoid arthritis.
B : A patient treated with the new medication for rheumatoid arthritis could sustain a joint injury without becoming aware of it.
C : Joint diseases other than rheumatoid arthritis would not be affected by the new medication.
D : The benefits to rheumatoid arthritis sufferer

In [20]:
def train_dp_to_prompt(td):
    prompt = ""
    prompt+= "Context: " + td['context'] + "\n"
    prompt+= "Question: " + td['question'] + "\n"
    prompt+= "Options:\n"
    for i, ans in enumerate(td['answers']):
        prompt+= Options[i]+' : ' + ans + "\n"
    prompt+= "Correct Answer: " + Options[td['label']] + "\n"
    prompt+= "<stop>\n"
    return prompt

In [21]:
def test_dp_to_prompt(td):
    prompt = ""
    prompt+= "Context: " + td['context'] + "\n"
    prompt+= "Question: " + td['question'] + "\n"
    prompt+= "Answer Options:\n"
    for i, ans in enumerate(td['answers']):
        prompt+= Options[i] + " : " + ans + "\n"
    return prompt

In [22]:
train_prompt = "\n".join([train_dp_to_prompt(train[i]) for i in train_ixs])
print(train_prompt)

Context: In rheumatoid arthritis, the body' s immune system misfunctions by attacking healthy cells in the joints causing the release of a hormone that in turn causes pain and swelling. This hormone is normally activated only in reaction to injury or infection. A new arthritis medication will contain a protein that inhibits the functioning of the hormone that causes pain and swelling in the joints.
Question: The statements above, if true, most strongly support which one of the following conclusions?
Options:
A : Unlike aspirin and other medications that reduce pain and swelling and that are currently available, the new medication would repair existing cell damage that had been caused by rheumatoid arthritis.
B : A patient treated with the new medication for rheumatoid arthritis could sustain a joint injury without becoming aware of it.
C : Joint diseases other than rheumatoid arthritis would not be affected by the new medication.
D : The benefits to rheumatoid arthritis sufferers of th

In [23]:
def gen_answer_direct(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages = [
            {"role": "system", "content": "You answer the question given similar to the examples given. Always pick the best of the four options given."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
        stop = ["<stop>"]
    )
    return response["choices"][0]["message"]["content"]

In [24]:
prompt = train_prompt + test_dp_to_prompt(val[0])
print(gen_answer_direct(prompt))

Correct Answer: A


In [73]:
prompt = train_prompt + test_dp_to_prompt(val[1])
print(gen_answer_direct(prompt))

Argument Structure:
Premise 1: It takes a particular talent to be a successful business manager.
Premise 2: Business courses can help people to solve management problems, but such courses can do so only for those people with managerial talent.
Claim: Such people should take business courses to acquire ideas that they can subsequently use to good advantage if management problems happen to arise.
Premise 1 and Premise 2 together support the Claim.

Explanation: The argument does not provide any information to support the idea that people who are helped by business courses in solving management problems also have managerial talent. Therefore, option A cannot be true.
Correct Answer: C


In [56]:
prompt = train_prompt + test_dp_to_prompt(val[2])
print(gen_answer_direct(prompt))

Argument Structure:
Premise 1: An action is morally good if it both achieves the agent's intended goal and benefits someone other than the agent.
Claim: The judgment that most closely conforms to the principle cited by the philosopher.
Premise 1 supports the Claim.

Explanation: Yolanda's action of taking her children to visit their grandfather both achieved her intended goal of making her children enjoy their vacation and benefited her children and grandfather. Hence, it conforms to the principle cited by the philosopher.
Correct Answer: B


In [32]:
STORE_FILE_NAME = "direct_ans_old.json"

In [33]:
def store_in_file_json(td, label, gen_ans, ans_str):
    with open(os.path.join(DATA_FOLDER, STORE_FILE_NAME), 'a') as f:
        json.dump({"id_string": td['id_string'], "question": td['question'], "label": label, "answer" : ans_str, "generated": gen_ans}, f)
        f.write("\n")

In [34]:
def load_stored_preds():
    stored_ans_json = []
    if os.path.exists(os.path.join(DATA_FOLDER, STORE_FILE_NAME)):
        stored_ans_json = [json.loads(line) for line in open(os.path.join(DATA_FOLDER, STORE_FILE_NAME)).readlines()]
    return stored_ans_json

In [35]:
def get_accuracy():
    stored_ans_json = load_stored_preds()
    if len(stored_ans_json) == 0:
        return 0
    correct = 0
    for js in stored_ans_json:
        if str(js['label']) == str(js['answer']):
            correct+=1
    
    print("Correct answers: ", correct, ", Total processed: ", len(stored_ans_json))
    return correct/len(stored_ans_json)

In [36]:
def extract_answer_from_gen_ans(gen_ans):
    return re.findall(r'Correct Answer: ([ABCD]).*', gen_ans)[0]

In [37]:
print(len(val[:5]))
val_set = val

5


In [38]:
start_ix = 0

stored_ans_json = load_stored_preds()
stored_ixs = [int(js['id_string'].split('_')[-1]) for js in stored_ans_json]
# if len(stored_ans_json) > 0:
#     start_ix = int(stored_ans_json[-1]['id_string'].split('_')[-1]) + 1
# print("Starting from index", start_ix)

labels = []
preds = []
corrects = 0
options = ['A', 'B', 'C', 'D']
for td in val_set[start_ix:]:
    ix = int(td['id_string'].split('_')[-1])
    if ix not in stored_ixs:
        try:
            label = options[td['label']]
            # print('Correct ans: ', label)
            prompt = train_prompt + test_dp_to_prompt(td)
            gen_ans = gen_answer_direct(prompt)
            ans_str = extract_answer_from_gen_ans(gen_ans)
            store_in_file_json(td, label, gen_ans, ans_str)
            # print('Predicted ans: ', ans)

            labels.append(label)
            preds.append(ans_str)
            if ans_str == label:
                corrects+=1
        except Exception as e:
            print(e)
            print('Error in: ', td['id_string'])
            print(gen_ans)
            print('\n')
            time.sleep(60)
            continue
        stored_ixs.append(ix)
print(get_accuracy())

Correct answers:  291 , Total processed:  500
0.582
